In [1]:
#1. Parsing HTML
from lxml import html
import re
import requests
import pandas as pd
from datetime import datetime

#get html and tree
html_page_link = 'webpage.html'
tree = html.parse(html_page_link).getroot()

painting_records = {
    'artist_name': None,
    'painting_name': None,
    'price_gbp': None,
    'price_us': None,
    'price_gbp_est': None,
    'price_us_est': None,
    'image_link': None,
    'sell_date': None
}

# Parse the artist name
# Newline are removed before assignent
for element in tree.iter('h1'):
    painting_records['artist_name'] = element.text_content().split('\r\n')[0]

# Parse painting name
for element in tree.iter('h2'):
    painting_records['painting_name'] = element.text_content().split('\r\n')[0]

# All prices should be parsed into integer
# Parse price GBP
painting_price_element_gbp = tree.get_element_by_id('main_center_0_lblPriceRealizedPrimary')
painting_price_content_gbp = painting_price_element_gbp.text_content()
painting_records['price_gbp'] = int(painting_price_content_gbp.split(' ')[1].replace(',', ''))

# Parse price US
painting_price_element_us = tree.get_element_by_id('main_center_0_lblPriceRealizedSecondary')
painting_price_content_us = painting_price_element_us.text_content()
painting_records['price_us'] = int(painting_price_content_us.split(' ')[1].replace(',', ''))

# Parse price GBP est
painting_price_element_gbp_est = tree.get_element_by_id('main_center_0_lblPriceEstimatedPrimary')
painting_price_content_gbp_est = painting_price_element_gbp_est.text_content()
painting_records['price_gbp_est'] = int(painting_price_content_gbp_est.split(' ')[1].replace(',', ''))

# Parse price US est
painting_price_element_us_est = tree.get_element_by_id('main_center_0_lblPriceEstimatedSecondary')
painting_price_content_us_est = painting_price_element_us_est.text_content()
painting_records['price_us_est'] = int(painting_price_content_us_est.split(' ')[1].replace(',', ''))

# Parse the image link
image_element = tree.get_element_by_id('imgLotImage')
painting_records['image_link'] = image_element.attrib['src']

# Parse the sell date
sell_date_element = tree.get_element_by_id('main_center_0_lblSaleDate')
sell_date_string = sell_date_element.text_content().split(',')[0]
painting_records['sell_date'] = datetime.strptime(sell_date_string, '%d %B %Y').date()

# Convert dict to pd dataframe
painting_records_dataframe = pd.DataFrame([painting_records])
painting_records_dataframe

,artist_name,painting_name,price_gbp,price_us,price_gbp_est,price_us_est,image_link,sell_date
0,Peter Doig (b. 1959),The Architect's Home in the Ravine,11282500,16370908,10000000,14509999,http://www.christies.com/lotfinderimages/D5973...,2016-02-11


In [2]:
# 2. Regex

import pandas as pd
import re
import numpy as np
dim_df = pd.read_csv("dim_df_correct.csv")
dim_df

,rawDim,height,width,depth
0,19×52cm,19.0,52.0,NaN
1,"50 x 66,4 cm",50.0,66.4,NaN
2,168.9 x 274.3 x 3.8 cm (66 1/2 x 108 x 1 1/2 in.),168.9,274.3,3.8
3,Sheet: 16 1/4 × 12 1/4 in. (41.3 × 31.1 cm) Im...,35.6,25.1,NaN
4,5 by 5in,12.7,12.7,NaN


In [3]:
########### reg expressions

# Preprocessing for special character ','
def replaceSpecialChar(dimension_string):
    dimension_string_seperated = dimension_string.split('x')
    for i, item in enumerate(dimension_string_seperated):
        if (',' in item) is True:
            for char in item:
                item = item.replace(',',".")
                dimension_string_seperated[i] = item
    return ' '.join(dimension_string_seperated)

#Preprocessing for fractions

dim_df['rawDim'] = dim_df['rawDim'].map( lambda x : replaceSpecialChar(x))

for dimString in dim_df['rawDim']:    
    # If any other content contains ',' then switch to a .
    for i, item in enumerate(dimString.split('x')):
        if (',' in item) is True:
            for char in item:
                item = item.replace(',',".")
                
    numbers = re.findall('-?\d+\.?\d*',dimString)
    try:
        print('Height: {} Width: {} Depth: {}'.format(np.float64(numbers[0]), np.float64(numbers[1]), np.float64(numbers[2])))
    except IndexError as e:
        if len(numbers) < 3: 
            print('Height: {} Width: {}'.format(np.float64(numbers[0]), np.float64(numbers[1])))


Height: 19.0 Width: 52.0
Height: 50.0 Width: 66.4
Height: 168.9 Width: 274.3 Depth: 3.8
Height: 16.0 Width: 1.0 Depth: 4.0
Height: 5.0 Width: 5.0


In [4]:
# 3. Web crawler
listings_link = 'https://www.bearspace.co.uk/purchase/'
res = requests.get(listings_link)
listings_page_content = html.fromstring(res.content)
gallery_records = {
    'url': [],
    'title': [],
    'media': [0] *20 ,
    'height': [],
    'width': [],
    'price_gbp': []
}

for element in listings_page_content.iter('a'):
    for item in element.items():
        if 'class' in item:
            if item[1] == '_2zTHN _2AHc6':
                # Obtain URL from the 'a' tags
                url = element.attrib['href']
                gallery_records['url'].append(url)
                
                # Scrape the product page
                product_page_content = html.fromstring(requests.get(url).content)
                
                # Obtain the price of the gallery
                for product_page_div in product_page_content.iter('div'):
                    for item in product_page_div.items():
                        if 'class' in item:
                            if item[1] == '_2sFaY':
                                for i, product_page_span in enumerate(product_page_div.iter('span')):
                                    if i == 0:
                                        gallery_records['price_gbp'].append(product_page_span.text_content())
                                    continue
                # Obtain title form the 'h3' tags
                for child_element in element.iter('h3'):
                    gallery_records['title'].append(child_element.text_content())

# For each URL scrape the product page and obtain the media, height and width
for i, url in enumerate(gallery_records['url']):
    # Scrape the product page
    product_page_content = html.fromstring(requests.get(url).content)
    
    for pd_element in product_page_content.iter('pre'):
        for pd_element_description in pd_element.iter('p'):

            # Obtain the dimensions and media from the first 'p' tag
            numbers = re.findall('-?\d+\.?\d*', pd_element_description.text_content())

            # If there are no dimension then p tag contains the media
            if len(numbers) == 2:
                gallery_records['height'].append(numbers[0])
                gallery_records['width'].append(numbers[1])
            # Description of painting will not contain numbers and spaces    
            elif len(numbers) == 0 and pd_element_description.text_content().isspace() is False:
                gallery_records['media'][i] = pd_element_description.text_content()


gallery_records_dataframe = pd.DataFrame(gallery_records)
gallery_records_dataframe

,url,title,media,height,width,price_gbp
0,https://www.bearspace.co.uk/product-page/overl...,Overland I by Jane Ward,"Archival Digital print on stretched canvas, fi...",90,120,"£1,400.00"
1,https://www.bearspace.co.uk/product-page/the-h...,The Hurricane by Lucy Baker,"Oil paint, spray paint & gold leaf on board",110,130,"£1,600.00"
2,https://www.bearspace.co.uk/product-page/amazo...,Amazons by Lucy Baker,"Oil paint, spray paint & gold leaf on board",110,130,"£1,600.00"
3,https://www.bearspace.co.uk/product-page/off-r...,OFF ROAD by Lucy Baker,"Oil paint, spray paint & gold leaf on board",30,40,£340.00
4,https://www.bearspace.co.uk/product-page/hurri...,Hurricane by Lucy Baker,Oil paint & spray paint on board,30,40,£340.00
5,https://www.bearspace.co.uk/product-page/anoth...,Another Sunset by Lucy Baker,"Oil paint, spray paint & gold leaf on board",55,71,£340.00
6,https://www.bearspace.co.uk/product-page/copy-...,Meld 13 by Vic Wright,Photo credit: Laura Hutchinson,16,16,£440.00
7,https://www.bearspace.co.uk/product-page/kin-2...,Kin 2 by Vic Wright,Photo credit: Laura Hutchinson,18,15,£440.00
8,https://www.bearspace.co.uk/product-page/kin-b...,Kin by Vic Wright,Photo credit: Laura Hutchinson,16,14,£440.00
9,https://www.bearspace.co.uk/product-page/morni...,Morning Came by Dominic Bradnum,Photo credit: Steve Speller,40,50,£640.00


In [5]:
# 4. Data
# Task 1 : Describe inner join, left join, right join, full join.7
print('Inner Join')
print('This is the combination of rows in two tables by using a  coloumns existing in two seperate tables')
print('Inner join combines both table based on the condition that there is a matching coloumn from both table, the resulting table will contain rows that are only present in both tables')
print('\n')

print('Left Join')
print('This works by combining two tables bassed on a condition that matches the two tables, also rows that are not matched from the table placed on the left of the clause will be present but will have null values if no matching row in the second table is found')
print('Left join takes all content from the table placed on the left regardsless of if the condiiton is met, and also includes content from both tables that match the condition')
print('\n')

print('Right Join')
print('This works by combining two tables bassed on a condition that matches the two tables, also rows that are not matched from the table placed on the right will be present but will have null values if no matching row in the second table is found')
print('The right join combined the two tables based on matching conditiona and also includes the rows from the table placed on the right of the clause, even though there are no matches')
print('\n')

print('Full Join')
print('This is a combination of both right join and left join and the resulting table will return all rows that are matched or unmatched based on condition from both sides of the query statement')


Inner Join
This is the combination of rows in two tables by using a  coloumns existing in two seperate tables
Inner join combines both table based on the condition that there is a matching coloumn from both table, the resulting table will contain rows that are only present in both tables


Left Join
This works by combining two tables bassed on a condition that matches the two tables, also rows that are not matched from the table placed on the left of the clause will be present but will have null values if no matching row in the second table is found
Left join takes all content from the table placed on the left regardsless of if the condiiton is met, and also includes content from both tables that match the condition


Right Join
This works by combining two tables bassed on a condition that matches the two tables, also rows that are not matched from the table placed on the right will be present but will have null values if no matching row in the second table is found
The right join comb

In [6]:
import pandasql as ps

flights = pd.read_csv("flights.csv")
airports = pd.read_csv("airports.csv")
weather = pd.read_csv("weather.csv")
airlines = pd.read_csv("airlines.csv")

# Add full airline name to the flights dataframe and show the arr_time, origin, dest and the name of the airline.
query1 = """SELECT flights.arr_time, flights.origin, flights.dest, flights.flight, flights.distance, airlines.name FROM airlines LEFT JOIN flights ON airlines.carrier = flights.carrier"""
# save query1 as dataframe
new_data_frame = ps.sqldf(query1, locals())

# Filter resulting data.frame to include only flights containing the word JetBlue
query2 = """SELECT * FROM new_data_frame WHERE name LIKE '%JetBlue%'"""
# save query 1 as dataframe
new_data_frame = ps.sqldf(query2, locals())

# Summarise the total number of flights by origin in ascending.
query3 = """SELECT origin, sum(flight) as numFlight FROM new_data_frame GROUP BY origin ORDER BY origin ASC"""
# save query 3 as dataframe
new_data_frame = ps.sqldf(query3, locals())
print(new_data_frame)
print("Might be missing the planes csv file")
# Filter resulting data.frame to return only origins with more than 10,000 flights.F
query4 = """SELECT * FROM new_data_frame WHERE flights > 10000"""
# save query 4 as dataframe
# # print query4

  origin  numFlight
0    EWR      850.0
1    JFK     2730.0
2    LGA      371.0
Might be missing the planes csv file
